In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint as pp
from collections import defaultdict

plt.style.use('seaborn')

In [2]:
df = pd.read_json('data/anime_content.json', lines=True)

In [3]:
df.head()

,_id,errors,data
0,{'$oid': '5ed06d2b64d930385059a26d'},"[{'message': 'Not Found.', 'status': 404, 'loc...",{'Media': None}
1,{'$oid': '5ed06d3564d930385059a26e'},NaN,"{'Media': {'id': 10161, 'title': {'romaji': 'N..."
2,{'$oid': '5ed06d3f64d930385059a26f'},NaN,"{'Media': {'id': 99726, 'title': {'romaji': 'N..."
3,{'$oid': '5ed06d4964d930385059a270'},NaN,"{'Media': {'id': 98526, 'title': {'romaji': 'R..."
4,{'$oid': '5ed06d5364d930385059a271'},NaN,"{'Media': {'id': 101240, 'title': {'romaji': '..."


In [4]:
# Some api calls returned None
no_media = []
for row in df['data']:
    if row['Media'] == None:
        no_media.append(True)
    else:
        no_media.append(False)

In [5]:
len(df[no_media])

48

In [6]:
df = df[~np.array(no_media)]

In [7]:
df[~np.array(df['errors'].isna())]

,_id,errors,data


In [8]:
# No remaining rows contain errors. Column is removed.
df = df[['_id', 'data']]

In [9]:
df.head()

,_id,data
1,{'$oid': '5ed06d3564d930385059a26e'},"{'Media': {'id': 10161, 'title': {'romaji': 'N..."
2,{'$oid': '5ed06d3f64d930385059a26f'},"{'Media': {'id': 99726, 'title': {'romaji': 'N..."
3,{'$oid': '5ed06d4964d930385059a270'},"{'Media': {'id': 98526, 'title': {'romaji': 'R..."
4,{'$oid': '5ed06d5364d930385059a271'},"{'Media': {'id': 101240, 'title': {'romaji': '..."
5,{'$oid': '5ed06d5e64d930385059a272'},"{'Media': {'id': 966, 'title': {'romaji': 'Cra..."


In [10]:
df.tail()

,_id,data
14473,{'$oid': '5ed2ab4364d930385059daf6'},"{'Media': {'id': 99916, 'title': {'romaji': 'A..."
14474,{'$oid': '5ed2ab4d64d930385059daf7'},"{'Media': {'id': 101283, 'title': {'romaji': '..."
14475,{'$oid': '5ed2ab5864d930385059daf8'},"{'Media': {'id': 101633, 'title': {'romaji': '..."
14476,{'$oid': '5ed2ab6264d930385059daf9'},"{'Media': {'id': 21742, 'title': {'romaji': 'K..."
14477,{'$oid': '5ed2ab6c64d930385059dafa'},"{'Media': {'id': 101089, 'title': {'romaji': '..."


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14430 entries, 1 to 14477
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     14430 non-null  object
 1   data    14430 non-null  object
dtypes: object(2)
memory usage: 338.2+ KB


In [12]:
df['data'][1]['Media']

{'id': 10161,
 'title': {'romaji': 'No.6',
  'english': 'No.6',
  'native': 'NO.6 ナンバー・シックス',
  'userPreferred': 'No.6'},
 'startDate': {'year': 2011, 'month': 7, 'day': 8},
 'endDate': {'year': 2011, 'month': 9, 'day': 16},
 'season': 'SUMMER',
 'seasonYear': 2011,
 'type': 'ANIME',
 'format': 'TV',
 'status': 'FINISHED',
 'episodes': 11,
 'duration': 23,
 'chapters': None,
 'volumes': None,
 'isAdult': False,
 'genres': ['Action', 'Sci-Fi', 'Drama'],
 'tags': [{'name': 'Dystopian', 'rank': 92, 'category': 'Setting-Time'},
  {'name': "Boys' Love", 'rank': 70, 'category': 'Theme-Romance'},
  {'name': 'Shoujo', 'rank': 70, 'category': 'Demographic'},
  {'name': 'Cyberpunk', 'rank': 66, 'category': 'Theme-Sci-Fi'},
  {'name': 'Crossdressing', 'rank': 50, 'category': 'Cast-Traits'},
  {'name': 'Yandere', 'rank': 44, 'category': 'Cast-Traits'}],
 'isLicensed': True,
 'averageScore': 71,
 'popularity': 27608,
 'source': 'OTHER',
 'countryOfOrigin': 'JP',
 'staff': {'edges': [{'id': 12992,
 

In [13]:
# for show in df['data'][:5]:
#     print('anilist_id', show['Media']['id'])
#     print('title_romaji', show['Media']['title']['romaji'])
#     print('title_english', show['Media']['title']['english'])
#     print('title_native', show['Media']['title']['native'])
#     print('title_userPreferred', show['Media']['title']['userPreferred'])
#     print('start_date', pd.to_datetime(str(show['Media']['startDate']['year']) + '-' +
#                          str(show['Media']['startDate']['month']) + '-' +
#                          str(show['Media']['startDate']['day'])).date())
#     print('end_date', pd.to_datetime(str(show['Media']['endDate']['year']) + '-' +
#                          str(show['Media']['endDate']['month']) + '-' +
#                          str(show['Media']['endDate']['day'])).date())
#     print('season', show['Media']['season'])
#     print('type', show['Media']['type'])
#     print('format', show['Media']['format'])
#     print('status', show['Media']['status'])
#     print('episodes', show['Media']['episodes'])
#     print('duration', show['Media']['duration'])
#     print('chapters', show['Media']['chapters'])
#     print('volumes', show['Media']['volumes'])
#     print('isAdult', show['Media']['isAdult'])
#     print('genres', show['Media']['genres'])
#     print('type', show['Media']['tags'])

    
#     print('------------------')

In [14]:
media_df = pd.DataFrame(list(df['data']))

In [15]:
media_df.head()

,Media
0,"{'id': 10161, 'title': {'romaji': 'No.6', 'eng..."
1,"{'id': 99726, 'title': {'romaji': 'Net-juu no ..."
2,"{'id': 98526, 'title': {'romaji': 'Robomasters..."
3,"{'id': 101240, 'title': {'romaji': 'Dokidoki L..."
4,"{'id': 966, 'title': {'romaji': 'Crayon Shin-c..."


In [16]:
media_df = pd.DataFrame(list(media_df['Media']))

In [17]:
media_df.head()

,id,title,startDate,endDate,season,seasonYear,type,format,status,episodes,...,genres,tags,isLicensed,averageScore,popularity,source,countryOfOrigin,staff,studios,characters
0,10161,"{'romaji': 'No.6', 'english': 'No.6', 'native'...","{'year': 2011, 'month': 7, 'day': 8}","{'year': 2011, 'month': 9, 'day': 16}",SUMMER,2011.0,ANIME,TV,FINISHED,11.0,...,"[Action, Sci-Fi, Drama]","[{'name': 'Dystopian', 'rank': 92, 'category':...",True,71.0,27608,OTHER,JP,"{'edges': [{'id': 12992, 'node': {'id': 101012...","{'edges': [{'id': 11804, 'node': {'id': 4, 'na...","{'edges': [{'id': 4370, 'role': 'SUPPORTING', ..."
1,99726,"{'romaji': 'Net-juu no Susume', 'english': 'Re...","{'year': 2017, 'month': 10, 'day': 6}","{'year': 2017, 'month': 12, 'day': 8}",FALL,2017.0,ANIME,TV,FINISHED,10.0,...,"[Comedy, Fantasy, Romance, Adventure, Slice of...","[{'name': 'Video Games', 'rank': 95, 'category...",True,75.0,41540,MANGA,JP,"{'edges': [{'id': 99978, 'node': {'id': 110641...","{'edges': [{'id': 17713, 'node': {'id': 6101, ...","{'edges': [{'id': 149462, 'role': 'MAIN', 'nod..."
2,98526,"{'romaji': 'Robomasters The Animated Series', ...","{'year': 2017, 'month': 10, 'day': 13}","{'year': 2017, 'month': 11, 'day': 17}",FALL,2017.0,ANIME,TV,FINISHED,6.0,...,"[Action, Drama]","[{'name': 'Real Robot', 'rank': 60, 'category'...",True,58.0,1901,ORIGINAL,JP,"{'edges': [{'id': 93476, 'node': {'id': 107517...","{'edges': [{'id': 17375, 'node': {'id': 6154, ...",{'edges': []}
3,101240,"{'romaji': 'Dokidoki Little Ooya-san', 'englis...","{'year': 2018, 'month': 5, 'day': 25}","{'year': 2019, 'month': 12, 'day': 27}",SPRING,2018.0,ANIME,OVA,FINISHED,6.0,...,[Hentai],"[{'name': 'Age Gap', 'rank': 86, 'category': '...",True,67.0,1228,VIDEO_GAME,JP,{'edges': []},"{'edges': [{'id': 18136, 'node': {'id': 6155, ...","{'edges': [{'id': 151698, 'role': 'MAIN', 'nod..."
4,966,"{'romaji': 'Crayon Shin-chan', 'english': 'Shi...","{'year': 1992, 'month': 4, 'day': 13}","{'year': None, 'month': None, 'day': None}",SPRING,1992.0,ANIME,TV,RELEASING,NaN,...,"[Comedy, Ecchi, Slice of Life]","[{'name': 'Episodic', 'rank': 85, 'category': ...",True,73.0,4681,MANGA,JP,"{'edges': [{'id': 966, 'node': {'id': 95148, '...","{'edges': [{'id': 2601, 'node': {'id': 55, 'na...","{'edges': [{'id': 2281, 'role': 'SUPPORTING', ..."


In [18]:
media_df['id']

0         10161
1         99726
2         98526
3        101240
4           966
          ...  
14425     99916
14426    101283
14427    101633
14428     21742
14429    101089
Name: id, Length: 14430, dtype: int64

In [36]:
tags_df = pd.DataFrame(list(media_df['tags']))
tags_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,"{'name': 'Dystopian', 'rank': 92, 'category': ...","{'name': 'Boys' Love', 'rank': 70, 'category':...","{'name': 'Shoujo', 'rank': 70, 'category': 'De...","{'name': 'Cyberpunk', 'rank': 66, 'category': ...","{'name': 'Crossdressing', 'rank': 50, 'categor...","{'name': 'Yandere', 'rank': 44, 'category': 'C...",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"{'name': 'Video Games', 'rank': 95, 'category'...","{'name': 'Virtual World', 'rank': 92, 'categor...","{'name': 'Hikikomori', 'rank': 88, 'category':...","{'name': 'Primarily Adult Cast', 'rank': 83, '...","{'name': 'Female Protagonist', 'rank': 80, 'ca...","{'name': 'Parody', 'rank': 70, 'category': 'Th...","{'name': 'Gender Bending', 'rank': 55, 'catego...","{'name': 'Reverse Harem', 'rank': 20, 'categor...",None,None,...,None,None,None,None,None,None,None,None,None,None
2,"{'name': 'Real Robot', 'rank': 60, 'category':...","{'name': 'Robots', 'rank': 60, 'category': 'Ca...","{'name': 'College', 'rank': 20, 'category': 'S...",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [37]:
tags_df.iloc[0,0]

{'name': 'Dystopian', 'rank': 92, 'category': 'Setting-Time'}

In [38]:
tags_df.iloc[0,0]['category']

'Setting-Time'

In [59]:
tag_categories = {}
for idx, row in tags_df.iterrows():
    for tag in row:
        if tag == None:
            continue
        tag_categories.setdefault(tag['category'], []).append(tag['name'])

In [61]:
pp.pprint(tag_categories)

{'': ['Reformation'],
 'Cast-Main Cast': ['Primarily Adult Cast',
                    'Female Protagonist',
                    'Male Protagonist',
                    'Male Protagonist',
                    'Male Protagonist',
                    'Ensemble Cast',
                    'Male Protagonist',
                    'Ensemble Cast',
                    'Female Protagonist',
                    'Male Protagonist',
                    'Male Protagonist',
                    'Male Protagonist',
                    'Male Protagonist',
                    'Primarily Adult Cast',
                    'Male Protagonist',
                    'Primarily Male Cast',
                    'Male Protagonist',
                    'Ensemble Cast',
                    'Primarily Adult Cast',
                    'Ensemble Cast',
                    'Male Protagonist',
                    'Male Protagonist',
                    'Primarily Adult Cast',
                    'Male Protagonist',
       

                    'Primarily Female Cast',
                    'Primarily Adult Cast',
                    'Primarily Female Cast',
                    'Female Protagonist',
                    'Male Protagonist',
                    'Primarily Male Cast',
                    'Female Protagonist',
                    'Primarily Female Cast',
                    'Primarily Child Cast',
                    'Ensemble Cast',
                    'Primarily Female Cast',
                    'Male Protagonist',
                    'Primarily Male Cast',
                    'Ensemble Cast',
                    'Male Protagonist',
                    'Primarily Adult Cast',
                    'Ensemble Cast',
                    'Primarily Child Cast',
                    'Male Protagonist',
                    'Ensemble Cast',
                    'Female Protagonist',
                    'Female Protagonist',
                    'Primarily Female Cast',
                    'Female Protagoni

                    'Ensemble Cast',
                    'Primarily Female Cast',
                    'Male Protagonist',
                    'Female Protagonist',
                    'Female Protagonist',
                    'Primarily Female Cast',
                    'Primarily Female Cast',
                    'Female Protagonist',
                    'Female Protagonist',
                    'Female Protagonist',
                    'Primarily Male Cast',
                    'Primarily Female Cast',
                    'Primarily Adult Cast',
                    'Female Protagonist',
                    'Primarily Female Cast',
                    'Male Protagonist',
                    'Male Protagonist',
                    'Primarily Male Cast',
                    'Primarily Female Cast',
                    'Office Lady',
                    'Ensemble Cast',
                    'Primarily Female Cast',
                    'Male Protagonist',
                    'Female Protag

                 'Tsundere',
                 'Aliens',
                 'Gods',
                 'Teacher',
                 'Robots',
                 'Tsundere',
                 'Twins',
                 'Ghost',
                 'Gods',
                 'Demons',
                 'Amnesia',
                 'Yandere',
                 'Demons',
                 'Aliens',
                 'Shrine Maiden',
                 'Age Regression',
                 'Demons',
                 'Aliens',
                 'Elf',
                 'Vampire',
                 'Werewolf',
                 'Nudity',
                 'Demons',
                 'Monster Girl',
                 'Dragons',
                 'Maids',
                 'Tsundere',
                 'Crossdressing',
                 'Teacher',
                 'Aliens',
                 'Demons',
                 'Age Regression',
                 'Detective',
                 'Robots',
                 'Aliens',
            

                 'Detective',
                 'Werewolf',
                 'Witch',
                 'Zombie',
                 'Demons',
                 'Gods',
                 'Tsundere',
                 'Tsundere',
                 'Robots',
                 'Witch',
                 'Elf',
                 'Demons',
                 'Robots',
                 'Tsundere',
                 'Amnesia',
                 'Nudity',
                 'Demons',
                 'Witch',
                 'Delinquents',
                 'Aliens',
                 'Kemonomimi',
                 'Nekomimi',
                 'Ghost',
                 'Dissociative Identities',
                 'Crossdressing',
                 'Vampire',
                 'Detective',
                 'Demons',
                 'Amnesia',
                 'Nudity',
                 'Nudity',
                 'Kemonomimi',
                 'Demons',
                 'Skeleton',
                 'Tsundere',
    

...

                   'Body Swapping',
                   'Magic',
                   'Magic',
                   'Body Swapping',
                   'Super Power',
                   'Kaiju',
                   'Mythology',
                   'Magic',
                   'Magic',
                   'Henshin',
                   'Superhero',
                   'Super Power',
                   'Superhero',
                   'Henshin',
                   'Super Power',
                   'Magic',
                   'Mythology',
                   'Magic',
                   'Mythology',
                   'Henshin',
                   'Kaiju',
                   'Magic',
                   'Isekai',
                   'Super Power',
                   'Magic',
                   'Fairy Tale',
                   'Magic',
                   'Super Power',
                   'Super Power',
                   'Henshin',
                   'Henshin',
                   'Magic',
                

                 'Politics',
                 'Gore',
                 'Crime',
                 'Animals',
                 'Crossover',
                 'Cosmic Horror',
                 'Chibi',
                 'LGBTQ Issues',
                 'Gore',
                 'Body Horror',
                 'Gore',
                 'Chibi',
                 'War',
                 'Meta',
                 'Survival',
                 'Environmental',
                 'Chibi',
                 'Philosophy',
                 'Crime',
                 'Gore',
                 'Meta',
                 'Meta',
                 'War',
                 'Medicine',
                 'Politics',
                 'Gender Bending',
                 'Chibi',
                 'Meta',
                 'War',
                 'Noir',
                 'Gore',
                 'Memory Manipulation',
                 'Drugs',
                 'Reincarnation',
                 'Software Development',
        

                 'Body Horror',
                 'War',
                 'Politics',
                 'Animals',
                 'Chibi',
                 'War',
                 'Educational',
                 'War',
                 'Gender Bending',
                 'Otaku Culture',
                 'Animals',
                 'Animals',
                 'Drugs',
                 'Crime',
                 'Animals',
                 'Body Horror',
                 'Environmental',
                 'Philosophy',
                 'Environmental',
                 'Philosophy',
                 'Chibi',
                 'Gender Bending',
                 'War',
                 'Otaku Culture',
                 'Meta',
                 'Crossover',
                 'Philosophy',
                 'Crossover',
                 'Chibi',
                 'Gore',
                 'Drugs',
                 'Economics',
                 'Politics',
                 'Otaku Culture',
         

                         'Trains',
                         'Trains',
                         'Aviation',
                         'Aviation',
                         'Ships',
                         'Motorcycles',
                         'Ships',
                         'Trains',
                         'Trains',
                         'Tanks',
                         'Trains',
                         'Aviation',
                         'Aviation',
                         'Mopeds',
                         'Ships',
                         'Ships',
                         'Aviation',
                         'Ships',
                         'Ships',
                         'Cars',
                         'Trains',
                         'Cars',
                         'Motorcycles',
                         'Ships',
                         'Mopeds',
                         'Trains',
                         'Ships',
                         'Trains',
             

                   'Age Gap',
                   'Love Triangle',
                   "Boys' Love",
                   'Harem',
                   'Love Triangle',
                   'Age Gap',
                   'Harem',
                   'Yuri',
                   'Reverse Harem',
                   'Harem',
                   'Love Triangle',
                   'Yuri',
                   'Harem',
                   "Boys' Love",
                   'Yuri',
                   'Harem',
                   "Boys' Love",
                   'Yuri',
                   'Yuri',
                   "Boys' Love",
                   "Teens' Love",
                   'Love Triangle',
                   'Harem',
                   'Harem',
                   "Boys' Love",
                   'Harem',
                   'Yuri',
                   'Reverse Harem',
                   'Love Triangle',
                   'Yuri',
                   "Boys' Love",
                   'Harem',
               

In [22]:
genres_df = pd.DataFrame(list(media_df['genres']))
genres_df.head()

,0,1,2,3,4,5,6
0,Action,Sci-Fi,Drama,None,None,None,None
1,Comedy,Fantasy,Romance,Adventure,Slice of Life,None,None
2,Action,Drama,None,None,None,None,None
3,Hentai,None,None,None,None,None,None
4,Comedy,Ecchi,Slice of Life,None,None,None,None


In [23]:
# genres_df['anilist_id'] = media_df['id']
# genres_df.head()

In [24]:
staff_df = pd.DataFrame(media_df['staff'])
staff_df.head()

,staff
0,"{'edges': [{'id': 12992, 'node': {'id': 101012..."
1,"{'edges': [{'id': 99978, 'node': {'id': 110641..."
2,"{'edges': [{'id': 93476, 'node': {'id': 107517..."
3,{'edges': []}
4,"{'edges': [{'id': 966, 'node': {'id': 95148, '..."


In [25]:
staff_df = staff_df['staff'].apply(pd.Series)

In [26]:
staff_df = staff_df['edges'].apply(pd.Series)
staff_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,"{'id': 12992, 'node': {'id': 101012, 'name': {...","{'id': 18470, 'node': {'id': 101891, 'name': {...","{'id': 18867, 'node': {'id': 102008, 'name': {...","{'id': 24909, 'node': {'id': 103541, 'name': {...","{'id': 31679, 'node': {'id': 105635, 'name': {...","{'id': 35470, 'node': {'id': 106631, 'name': {...","{'id': 35471, 'node': {'id': 106631, 'name': {...","{'id': 37601, 'node': {'id': 107510, 'name': {...","{'id': 37602, 'node': {'id': 107510, 'name': {...","{'id': 38407, 'node': {'id': 107859, 'name': {...",...,"{'id': 38723, 'node': {'id': 108677, 'name': {...","{'id': 38724, 'node': {'id': 108677, 'name': {...","{'id': 38730, 'node': {'id': 108679, 'name': {...","{'id': 43802, 'node': {'id': 111539, 'name': {...","{'id': 44106, 'node': {'id': 111683, 'name': {...","{'id': 44422, 'node': {'id': 111749, 'name': {...","{'id': 45259, 'node': {'id': 112641, 'name': {...","{'id': 49442, 'node': {'id': 116017, 'name': {...","{'id': 50263, 'node': {'id': 116607, 'name': {...","{'id': 89006, 'node': {'id': 121209, 'name': {..."
1,"{'id': 99978, 'node': {'id': 110641, 'name': {...","{'id': 99979, 'node': {'id': 107576, 'name': {...","{'id': 101442, 'node': {'id': 123305, 'name': ...","{'id': 103313, 'node': {'id': 95936, 'name': {...","{'id': 103314, 'node': {'id': 118478, 'name': ...","{'id': 103315, 'node': {'id': 96105, 'name': {...","{'id': 103316, 'node': {'id': 118959, 'name': ...","{'id': 103318, 'node': {'id': 123959, 'name': ...","{'id': 103319, 'node': {'id': 123960, 'name': ...","{'id': 103321, 'node': {'id': 123961, 'name': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'id': 93476, 'node': {'id': 107517, 'name': {...","{'id': 93477, 'node': {'id': 118495, 'name': {...","{'id': 96834, 'node': {'id': 118453, 'name': {...","{'id': 96835, 'node': {'id': 120009, 'name': {...","{'id': 96836, 'node': {'id': 101573, 'name': {...","{'id': 96837, 'node': {'id': 122176, 'name': {...","{'id': 96838, 'node': {'id': 122177, 'name': {...","{'id': 118088, 'node': {'id': 131854, 'name': ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# staff_df['anilist_id'] = media_df['id']
# staff_df.head(3)

In [28]:
studios_df = pd.DataFrame(media_df['studios'])
studios_df.head()

,studios
0,"{'edges': [{'id': 11804, 'node': {'id': 4, 'na..."
1,"{'edges': [{'id': 17713, 'node': {'id': 6101, ..."
2,"{'edges': [{'id': 17375, 'node': {'id': 6154, ..."
3,"{'edges': [{'id': 18136, 'node': {'id': 6155, ..."
4,"{'edges': [{'id': 2601, 'node': {'id': 55, 'na..."


In [29]:
studios_df = studios_df['studios'].apply(pd.Series)
studios_df.head(3)

,edges
0,"[{'id': 11804, 'node': {'id': 4, 'name': 'BONE..."
1,"[{'id': 17713, 'node': {'id': 6101, 'name': 'S..."
2,"[{'id': 17375, 'node': {'id': 6154, 'name': 'D..."


In [30]:
studios_df = studios_df['edges'].apply(pd.Series)
studios_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11
0,"{'id': 11804, 'node': {'id': 4, 'name': 'BONES...","{'id': 11805, 'node': {'id': 17, 'name': 'Anip...","{'id': 11806, 'node': {'id': 169, 'name': 'Fuj...","{'id': 11807, 'node': {'id': 376, 'name': 'Sen...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'id': 17713, 'node': {'id': 6101, 'name': 'Si...","{'id': 17899, 'node': {'id': 464, 'name': 'fly...","{'id': 17900, 'node': {'id': 6186, 'name': 'co...","{'id': 21501, 'node': {'id': 6421, 'name': 'Cr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'id': 17375, 'node': {'id': 6154, 'name': 'Da...","{'id': 17558, 'node': {'id': 3, 'name': 'GONZO...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# studios_df['anilist_id'] = media_df['id']
# studios_df.head(3)

In [32]:
characters_df = pd.DataFrame(media_df['characters'])
characters_df.head()

,characters
0,"{'edges': [{'id': 4370, 'role': 'SUPPORTING', ..."
1,"{'edges': [{'id': 149462, 'role': 'MAIN', 'nod..."
2,{'edges': []}
3,"{'edges': [{'id': 151698, 'role': 'MAIN', 'nod..."
4,"{'edges': [{'id': 2281, 'role': 'SUPPORTING', ..."


In [33]:
characters_df = characters_df['characters'].apply(pd.Series)
characters_df.head()

,edges
0,"[{'id': 4370, 'role': 'SUPPORTING', 'node': {'..."
1,"[{'id': 149462, 'role': 'MAIN', 'node': {'id':..."
2,[]
3,"[{'id': 151698, 'role': 'MAIN', 'node': {'id':..."
4,"[{'id': 2281, 'role': 'SUPPORTING', 'node': {'..."


In [34]:
characters_df = characters_df['edges'].apply(pd.Series)
characters_df.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,"{'id': 4370, 'role': 'SUPPORTING', 'node': {'i...","{'id': 17879, 'role': 'SUPPORTING', 'node': {'...","{'id': 20798, 'role': 'SUPPORTING', 'node': {'...","{'id': 21575, 'role': 'SUPPORTING', 'node': {'...","{'id': 21964, 'role': 'MAIN', 'node': {'id': 4...","{'id': 37533, 'role': 'SUPPORTING', 'node': {'...","{'id': 40195, 'role': 'SUPPORTING', 'node': {'...","{'id': 47374, 'role': 'SUPPORTING', 'node': {'...","{'id': 50106, 'role': 'SUPPORTING', 'node': {'...","{'id': 53548, 'role': 'SUPPORTING', 'node': {'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'id': 149462, 'role': 'MAIN', 'node': {'id': ...","{'id': 149463, 'role': 'SUPPORTING', 'node': {...","{'id': 149464, 'role': 'SUPPORTING', 'node': {...","{'id': 149465, 'role': 'MAIN', 'node': {'id': ...","{'id': 149467, 'role': 'SUPPORTING', 'node': {...","{'id': 149468, 'role': 'SUPPORTING', 'node': {...","{'id': 149469, 'role': 'SUPPORTING', 'node': {...","{'id': 149470, 'role': 'SUPPORTING', 'node': {...","{'id': 150118, 'role': 'MAIN', 'node': {'id': ...","{'id': 150119, 'role': 'MAIN', 'node': {'id': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'id': 151698, 'role': 'MAIN', 'node': {'id': ...","{'id': 151699, 'role': 'MAIN', 'node': {'id': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'id': 2281, 'role': 'SUPPORTING', 'node': {'i...","{'id': 2770, 'role': 'SUPPORTING', 'node': {'i...","{'id': 8172, 'role': 'MAIN', 'node': {'id': 33...","{'id': 11061, 'role': 'SUPPORTING', 'node': {'...","{'id': 17504, 'role': 'SUPPORTING', 'node': {'...","{'id': 17505, 'role': 'MAIN', 'node': {'id': 2...","{'id': 20704, 'role': 'SUPPORTING', 'node': {'...","{'id': 27174, 'role': 'SUPPORTING', 'node': {'...","{'id': 28782, 'role': 'SUPPORTING', 'node': {'...","{'id': 29103, 'role': 'SUPPORTING', 'node': {'...",...,"{'id': 38375, 'role': 'SUPPORTING', 'node': {'...","{'id': 38468, 'role': 'SUPPORTING', 'node': {'...","{'id': 42278, 'role': 'SUPPORTING', 'node': {'...","{'id': 42632, 'role': 'SUPPORTING', 'node': {'...","{'id': 46210, 'role': 'SUPPORTING', 'node': {'...","{'id': 47993, 'role': 'MAIN', 'node': {'id': 7...","{'id': 74614, 'role': 'SUPPORTING', 'node': {'...",NaN,NaN,NaN
